## 1. Import Libraries

In [24]:
import os
import re
import pickle
import string
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

## 2. Download Stopwords

In [25]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) - {'not', 'no', 'never', 'nothing', 'nowhere'}

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 3. Data Upload

In [26]:
train_data = pd.read_csv('/content/twitter_data/twitter_training.csv', names=['id','entity','sentiment','text'])
test_data  = pd.read_csv('/content/twitter_data/twitter_validation.csv', names=['id','entity','sentiment','text'])

print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)

Train shape: (74682, 4)
Test shape: (1000, 4)


## 4. Data Cleaning

In [27]:
train_data.drop_duplicates(inplace=True)
train_data.dropna(subset=['text'], inplace=True)
test_data.drop_duplicates(inplace=True)
test_data.dropna(subset=['text'], inplace=True)

In [28]:
train_data = train_data[train_data['sentiment'] != 'Irrelevant']
test_data = test_data[test_data['sentiment'] != 'Irrelevant']

In [29]:
positive_words = ['love', 'best', 'amazing', 'great', 'excellent', 'perfect', 'incredible', 'awesome', 'good', 'nice']
negative_words = ['hate', 'terrible', 'awful', 'sucks', 'horrible', 'worst', 'bad', 'disgusting', 'broken']

def is_misleading_neutral(text):
    text_lower = str(text).lower()
    return any(w in text_lower for w in positive_words) or any(w in text_lower for w in negative_words)

neutral_mask = (train_data['sentiment'] == 'Neutral') & train_data['text'].apply(is_misleading_neutral)
train_data = train_data[~neutral_mask]

## 5. Text Preprocessing


In [30]:
def clean_text(text):
    if not isinstance(text, str):
        text = ""
    text = text.lower()
    text = re.sub(r'@[\w]+', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return ' '.join(words)

train_data['cleaned_text'] = train_data['text'].apply(clean_text)
test_data['cleaned_text'] = test_data['text'].apply(clean_text)

## 6. Encode Labels

In [31]:
label_encoder = LabelEncoder()
train_data['label'] = label_encoder.fit_transform(train_data['sentiment'])
test_data['label'] = label_encoder.transform(test_data['sentiment'])

print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

{'Negative': np.int64(0), 'Neutral': np.int64(1), 'Positive': np.int64(2)}


## 7. Tokenization & Padding

In [32]:
max_words = 10000
max_len = 80

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data['cleaned_text'])

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['cleaned_text']), maxlen=max_len, padding='post')
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['cleaned_text']), maxlen=max_len, padding='post')

y_train = to_categorical(train_data['label'], num_classes=3)
y_test = to_categorical(test_data['label'], num_classes=3)

## 8. Build Model

In [33]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128),
    Bidirectional(LSTM(32, return_sequences=True)),
    Bidirectional(LSTM(16)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model.build(input_shape=(None, max_len))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 80, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ (None, 80, 64)         │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ (None, 32)             │        10,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,335,875 (5.10 MB)

 Trainable params: 1,335,875 (5.10 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    batch_size=64,
    epochs=10,
    validation_data=(X_train, y_train,),
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/10
868/868 ━━━━━━━━━━━━━━━━━━━━ 29s 29ms/step - accuracy: 0.6150 - loss: 0.8324 - val_accuracy: 0.8640 - val_loss: 0.3504
Epoch 2/10
868/868 ━━━━━━━━━━━━━━━━━━━━ 23s 27ms/step - accuracy: 0.8640 - loss: 0.3587 - val_accuracy: 0.9095 - val_loss: 0.2291
Epoch 3/10
868/868 ━━━━━━━━━━━━━━━━━━━━ 41s 27ms/step - accuracy: 0.9048 - loss: 0.2417 - val_accuracy: 0.9277 - val_loss: 0.1753
Epoch 4/10
868/868 ━━━━━━━━━━━━━━━━━━━━ 44s 31ms/step - accuracy: 0.9228 - loss: 0.1905 - val_accuracy: 0.9386 - val_loss: 0.1394
Epoch 5/10
868/868 ━━━━━━━━━━━━━━━━━━━━ 38s 27ms/step - accuracy: 0.9332 - loss: 0.1608 - val_accuracy: 0.9468 - val_loss: 0.1193
Epoch 6/10
868/868 ━━━━━━━━━━━━━━━━━━━━ 41s 27ms/step - accuracy: 0.9417 - loss: 0.1358 - val_accuracy: 0.9505 - val_loss: 0.1110
Epoch 7/10
868/868 ━━━━━━━━━━━━━━━━━━━━ 41s 27ms/step - accuracy: 0.9468 - loss: 0.1223 - val_accuracy: 0.9538 - val_loss: 0.0978
Epoch 8/10
868/868 ━━━━━━━━━━━━━━━━━━━━ 41s 27ms/step - accuracy: 0.9497 - loss: 0.1095 - 

## 9. Evaluate Model

In [35]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

Test Accuracy: 0.9155
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step

Classification Report:
              precision    recall  f1-score   support

    Negative       0.93      0.97      0.95       266
     Neutral       0.96      0.82      0.88       285
    Positive       0.87      0.96      0.91       277

    accuracy                           0.92       828
   macro avg       0.92      0.92      0.92       828
weighted avg       0.92      0.92      0.91       828



## 10. Save Model & Tokenizer

In [36]:
accuracy_score = 0.92
model_filename = f"sentiment_lstm_model_acc_{accuracy_score:.2f}.h5"
tokenizer_filename = f"tokenizer_acc_{accuracy_score:.2f}.pkl"
label_encoder_filename = f"label_encoder_acc_{accuracy_score:.2f}.pkl"

model.save(model_filename)

with open(tokenizer_filename, 'wb') as f:
    pickle.dump(tokenizer, f)

with open(label_encoder_filename, 'wb') as f:
    pickle.dump(label_encoder, f)

print(f"Model and tokenizer saved successfully with accuracy {accuracy_score:.2f}")

Model and tokenizer saved successfully with accuracy 0.92


In [40]:
sample_texts = [
    "I really love this product, it's amazing!",
    "The service was terrible and I will never come back",
    "It's okay, nothing special but not bad"
]

sample_cleaned = [clean_text(t) for t in sample_texts]
sample_seq = tokenizer.texts_to_sequences(sample_cleaned)
sample_pad = pad_sequences(sample_seq, maxlen=max_len, padding='post')

predictions = model.predict(sample_pad)
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

for i in range(len(sample_texts)):
    print(f"Text: {sample_texts[i]}")
    print(f"Predicted Sentiment: {predicted_labels[i]}")
    print("-" * 50)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Text: I really love this product, it's amazing!
Predicted Sentiment: Positive
--------------------------------------------------
Text: The service was terrible and I will never come back
Predicted Sentiment: Negative
--------------------------------------------------
Text: It's okay, nothing special but not bad
Predicted Sentiment: Positive
--------------------------------------------------
